In [1]:
# imports
import pandas as pandas
from Bio import SeqIO
from Bio.KEGG.REST import *
from Bio.KEGG.KGML import KGML_parser
from Bio.Graphics.KGML_vis import KGMLCanvas
from Bio.Graphics.ColorSpiral import ColorSpiral

from IPython.display import Image, HTML

import random

In [2]:
# copied from 

# A bit of code that will help us display the PDF output
def PDF(filename):
    return HTML('<iframe src=%s width=700 height=350></iframe>' % filename)

# A bit of helper code to shorten long text
def head(text, lines=10):
    """ Print the first lines lines of the passed text.
    """
    print('\n'.join(text.split('\n')[:lines] + ['[...]']))

## Task 1: KEGG and gene id mapping

Familiarize yourself with the KEGG Rest interface and how to access it with Biopyhton:

http://www.genome.jp/kegg/rest/keggapi.html

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

### Subtask 1.1 Extract gene lists for all (mouse) KEGG pathways and store them in a suitable Python data structure

In [3]:
# download mus musculus pathways
mus_musculus_pathways = kegg_list("pathway", "mmu").read()

# convert into DataFrame
# pathway line format:
# path:<id>\t<description>\n
df = pandas.DataFrame([line.replace(':', '\t', 1).split('\t') for line in mus_musculus_pathways.split('\n')], columns = ['Org', 'ID', 'Description'])

# remove last row containing 'NaN'
df = df[:-1]

# set index
df.set_index('ID', inplace=True)

# store pathway names in list
#musMusculusKEGG_pathwayNames = [line.split("\t") for line in mus.split("\n")]



In [79]:
df.head()

,Org,Description
ID,,
mmu00010,pathGenes,Glycolysis / Gluconeogenesis - Mus musculus (m...
mmu00020,pathGenes,Citrate cycle (TCA cycle) - Mus musculus (mous...
mmu00030,pathGenes,Pentose phosphate pathway - Mus musculus (mous...
mmu00040,pathGenes,Pentose and glucuronate interconversions - Mus...
mmu00051,pathGenes,Fructose and mannose metabolism - Mus musculus...


In [ ]:
# get genes of its corresponding pathway

def getGenes(entry):
    if 'GENE' in entry:
        en = entry.replace("COMPOUND", "GENE").split("GENE")[1].split("\n")
        return [e.split(';')[0].split()[-1:] for e in en]


gene_set = [getGenes(kegg_get(pathway).read()) for pathway in df.index]

In [ ]:
gene_df = pandas.DataFrame(gene_set, columns=['Genes'])

### Subtask 1.2: Save the KEGG gene sets as a gmt file after you made sure they have the proper gene ids with respect to your DE analysis

hints: 

http://www.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats

## Task 2: Gene Set Enrichment

### Subtask 2.1: Read in the csv file you produced during the Differential Expression module, extract a gene list (as a python list of gene symbols) from your favorite multiple correction column (and store it in a variable)

### Subtask 2.2: Perform gene set enrichment (Fisher's exact test or an hypergeometric test will do for our purposes) with the KEGG gene sets you extracted in Task 1 (you may want to store the results in a pandas dataframe and write them to csv)

hint:

https://genetrail2.bioinf.uni-sb.de/help?topic=set_level_statistics

### Subtask 2.3: Extract a list of significantly (at 0.05 significance) enriched KEGG pathways

## Task 3: KEGG map visualization

#### hint:

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

#### remark:

In real life you may want to use the R-based tool pathview: https://bioconductor.org/packages/release/bioc/html/pathview.html (if you insist you can also try to use r2py for using pathview from Python during the practical)

For Python (in addition to the Biopyhton module) https://github.com/idekerlab/py2cytoscape in combination with https://github.com/idekerlab/KEGGscape may be another alternative (in the future)

Generally speaking, it is always a good idea to pay attention also to other pathway databases like Reactome or WikiPathways ...

### Subtask 3.1: Pick some significantly enriched KEGG pathways of your choice from 2.3 and visualize them

### Subtask 3.2: Define a a suitable binary color scheme respresenting the fact whether a gene is significantly expressed or not

hint: 

http://www.rapidtables.com/web/color/RGB_Color.htm

### Subtask 3.3: Visualize the pathway(s) from 3.1 in such a way that the included genes have the corresponding color from 3.2 ( you may need to define a suitable mapping from single genes to what is actually shown in the pathway map...)